# Condensing our dataframe to fit our needs
To simplify our data to make it easier to analyze, we'll be limiting our data with a few different constraints:
- Years: 2014-2021
- Top 10 High Risk Airports
- Top 5 Airline Carriers
- Adding columns for visualizations

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats

In [21]:
# loading the refined data
df = pd.read_csv("../csv/Airline_Delay_Cause.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318017 entries, 0 to 318016
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   year                 318017 non-null  int64  
 1   month                318017 non-null  int64  
 2   carrier              318013 non-null  object 
 3   carrier_name         318013 non-null  object 
 4   airport              318014 non-null  object 
 5   airport_name         318017 non-null  object 
 6   arr_flights          317524 non-null  float64
 7   arr_del15            317285 non-null  float64
 8   carrier_ct           317525 non-null  float64
 9   weather_ct           317523 non-null  float64
 10  nas_ct               317529 non-null  float64
 11  security_ct          317529 non-null  float64
 12  late_aircraft_ct     317529 non-null  float64
 13  arr_cancelled        317529 non-null  float64
 14  arr_diverted         317527 non-null  float64
 15  arr_delay        

In [22]:
# checking for NULL flight counts
df[df['arr_flights'].isna()].info()

# handling NULL flights, but preserving the flights with valid cancellation, diverted, and delay information
df = df.dropna(
    subset=['arr_flights', 'arr_del15', 'arr_cancelled', 'arr_diverted'],
    how='all'
)
# any preserved flights will still have a 0 flight count
df['arr_flights'] = df['arr_flights'].fillna(0)

# handling NULL diverted flights
df['arr_diverted'] = df['arr_diverted'].fillna(0)

# handling NULL delays
df['security_delay'] = df['security_delay'].fillna(0)
df['carrier_delay'] = df['carrier_delay'].fillna(0)
delay_var = [
    'carrier_delay',
    'weather_delay',
    'nas_delay',
    'security_delay',
    'late_aircraft_delay'
]

# filling in missing delay time with sum of all delay variables
df['arr_delay'] = df['arr_delay'].fillna(df[delay_var].sum(axis=1))

# handling NULL counts
df['carrier_ct'] = df['carrier_ct'].fillna(0)
df['weather_ct'] = df['weather_ct'].fillna(0)
count_var = [
    'carrier_ct',
    'weather_ct',
    'nas_ct',
    'security_ct',
    'late_aircraft_ct',
]

# filling in missing delay counts with sum of all count variables
df['arr_del15'] = df['arr_del15'].fillna(df[count_var].sum(axis=1))

# handling NULL carriers
df = df.dropna(
    subset=['carrier', 'carrier_name'],
    how='all'
)

# handling missing airport cells
missing_airport_codes = df.loc[df['airport'].isna(), 'airport_name']

# creating a map of all unique airports and their airport codes
airport_groups = (
    df
    .dropna(subset=['airport', 'airport_name'])
    .groupby('airport_name')['airport']
    .unique()
)

# filling blank airport cells from the airport groups map
df['airport'] = df['airport'].fillna(
    df['airport_name'].map(airport_groups)
)

# splitting the string by punctuation (identified pattern in the airport_name column)
airport_names = df['airport_name'].str.split(",|:", expand=True)
airport_names.columns = ['city', 'state', 'airport_name_cleansed']

# inserting columns into main dataframe, after airport codes
df.insert(5, "city", airport_names['city'].str.strip())
df.insert(6, "state", airport_names['state'].str.strip())
df.insert(8, "airport_name_cleansed", airport_names['airport_name_cleansed'].str.strip())

# renaming airport to airport_code and airport_name to airport_full_name for clarity
df = df.rename(columns = {"airport": "airport_code", "airport_name": "airport_full_name"})
df["airport_code"] = df["airport_code"].astype(str)

# converting year and month into datetime type
df.insert(0, "year_month", pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-01'))

# limits our dataframe to years 2014-2021
df = df[(df['year'] <= 2021) & (df['year'] >= 2014)]
# verifies the values of what years are included
df['year'].value_counts()
# change the number if you want to check for a specific year's data
# print(df.loc[df['year'] == 2014])

# sorts by year, airport, and the sums of delayed flight counts, sorts by year and descending delayed flight counts, and groups by year, getting the top 6 airports per year within 2014-2021 and compiling into a list of unique airports, which comes out to the top 10 most delayed airports
delays_by_year = (df
    .groupby(["year", "airport_code"])["arr_del15"].sum()
    .reset_index()
    .sort_values(["year", "arr_del15"], ascending=[True, False])
    .groupby("year")
    .head(6)
)

top_10 = delays_by_year['airport_code'].unique()

df = df[df["airport_code"].isin(top_10)]

df['airport_code'].value_counts()

# grouping the carrier flights together by the sum of their delayed flight counts, and grabbing the top 5 carriers with the most delays
top_carriers = (df
    .groupby(['carrier'])["arr_del15"].sum()
    .reset_index()
    .head(5)
)

df = df[df["carrier"].isin(top_carriers['carrier'])]

df['carrier'].value_counts()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 493 entries, 2872 to 316944
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 493 non-null    int64  
 1   month                493 non-null    int64  
 2   carrier              493 non-null    object 
 3   carrier_name         493 non-null    object 
 4   airport              493 non-null    object 
 5   airport_name         493 non-null    object 
 6   arr_flights          0 non-null      float64
 7   arr_del15            5 non-null      float64
 8   carrier_ct           5 non-null      float64
 9   weather_ct           5 non-null      float64
 10  nas_ct               5 non-null      float64
 11  security_ct          5 non-null      float64
 12  late_aircraft_ct     5 non-null      float64
 13  arr_cancelled        5 non-null      float64
 14  arr_diverted         5 non-null      float64
 15  arr_delay            5 non-null      fl

# Adding various columns including delay_rate, avg_delay_min, and season

In [23]:
df["delay_rate"] = np.where(
    df["arr_flights"] > 0,
    df["arr_del15"] / df["arr_flights"],
    np.nan
)
df["delay_rate"] = df["delay_rate"] * 100

df["avg_delay_min"] = df["arr_delay"] / df["arr_flights"]

season_map = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Fall", 10: "Fall", 11: "Fall"
}

df["season"] = df["month"].map(season_map)

df["date"] = pd.to_datetime(
    df["year"].astype(str) + "-" + df["month"].astype(str) + "-01"
)

# We reduced the rows of the csv file by almost 90%. The new csv file will be saved under "delaydata_final.csv"

NOTE: I renamed airport to airport_code and airport_name to airport_full_name for clarity, as well as added city, state, and the airport_name_cleansed (which is just the airport itself).

In [24]:
df.to_csv('../data/delaydata_final.csv', index=False, encoding='utf-8')

In [25]:
print(df.columns.tolist())


['year_month', 'year', 'month', 'carrier', 'carrier_name', 'airport_code', 'city', 'state', 'airport_full_name', 'airport_name_cleansed', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'delay_rate', 'avg_delay_min', 'season', 'date']
